### Import Dependencies & initialise env variables

In [10]:
import os
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

load_dotenv()

True

In [11]:
pdf_doc_directory_path = os.getenv("pdf_doc_directory_path")
db_directory = os.getenv("db_directory")
collection_name = os.getenv("collection_name")
embedding_model_name = os.getenv("embedding_model_name")

### Initialising the client objects

In [12]:
# Loading the embedding model
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
# Loading the PDF directory loader
pdf_directory_loader = PyPDFDirectoryLoader(path=pdf_doc_directory_path)
# Loading text-splitter for splitting the docs into small chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
# Initialising chroma client to load the documents into the vector-store
chroma_client = Chroma(persist_directory=db_directory)

In [4]:
# Loading all the PDFs in the directory
docs = pdf_directory_loader.load()
print("Number of docs loaded from the directory", len(docs))

# Splitting the documents into small chunk
doc_splits = text_splitter.split_documents(docs)
print("Number of documents after splitting", len(doc_splits))

Number of docs loaded from the directory 350
Number of documents after splitting 742


In [ ]:
# Adding the document chunks to the Chroma DB vectore-store
vectorstore = chroma_client.from_documents(
    documents=doc_splits,
    embedding=embeddings,
    collection_name=collection_name,
    persist_directory=db_directory
)

### Test the results of vector-search

In [9]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vectorstore = Chroma(
    persist_directory=db_directory,
    embedding_function=embeddings,
    collection_name=collection_name
)

vectorstore.similarity_search_with_score("How to start Ford Figo?")

[(Document(id='64640133-7533-43eb-8b7e-5ed339109d95', metadata={'page_label': '1', 'creationdate': '2015-06-12T10:48:16+00:00', 'creator': 'Unknown', 'total_pages': 245, 'moddate': '2015-09-08T17:52:47+05:30', 'page': 0, 'title': 'Untitled', 'source': '..\\data\\documents\\Ford-figo-owners-manual.pdf', 'author': 'Unknown', 'producer': 'XEP 4.16 build 20090723', 'trapped': '/False'}, page_content="FORD FIGO Owner's Manual"),
  0.5034269094467163),
 (Document(id='b1c0a39b-7344-447c-9063-9c1b471246f6', metadata={'title': 'Untitled', 'page': 81, 'producer': 'XEP 4.16 build 20090723', 'page_label': '79', 'creator': 'Unknown', 'creationdate': '2015-06-12T10:48:16+00:00', 'total_pages': 245, 'moddate': '2015-09-08T17:52:47+05:30', 'source': '..\\data\\documents\\Ford-figo-owners-manual.pdf', 'author': 'Unknown', 'trapped': '/False'}, page_content='1. Fully depress the clutch pedal.\n2. Start the engine.\nVehicles with Automatic Transmission\nNote: Do not touch the accelerator pedal.\n1. Depre

In [9]:
# Creating a retriever object to fetch similary search
retriever = vectorstore.as_retriever(
    search_type="mmr", search_kwargs={"k": 5, "fetch_k": 50}
)

retriever.invoke("How to start Hunter 350?")

[Document(id='2d17bd55-6935-4288-ac9c-329f23c4deb2', metadata={'title': 'untitled', 'page': 53, 'total_pages': 105, 'producer': 'Acrobat Distiller 22.0 (Windows)', 'creationdate': '2022-07-25T17:10:43+05:30', 'creator': 'Adobe InDesign 17.3 (Windows)', 'moddate': '2022-07-26T13:09:10+05:30', 'page_label': '54', 'source': '..\\data\\documents\\hunter-350-dual-channel.pdf', 'trapped': '/False'}, page_content='Royal Enfield Hunter 35052\nSTARTING\nCAUTION\n\x84 In case the malfunction \nindicator does not \nturn “OFF”, get the \nmotorcycle checked \nthrough an Royal Enfield \nAuthorised Service \nCentre for rectification.\n\x84 Disengage clutch by \npulling in the clutch \nlever and hold it in \ndepressed condition.\n\x84 Push and hold electric \nstart switch until engine \nstarts for a maximum of \n5 s.\n\x84 Check the fuel level indicator in the cluster for \nadequate fuel in the fuel tank. In case the last bar \nis blinking continuously, it indicates low fuel level in \nthe tank. Pleas